In [3]:
from __future__ import annotations

import bz2
import json
from collections import defaultdict
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd


ROOT = (Path(__file__) if "__file__" in globals() else Path(".")).resolve().parents[0]
INDEXES_DIR = ROOT / "Aditional datasets" / "indexes"
CROSSWALKS_DIR = ROOT / "Aditional datasets" / "Cross-walks"
SJMM_JSONL_DIR = (
    ROOT
    / "swissubase_669_10_0"
    / "data"
    / "669_SJMM_Data_SUF_v10.0"
    / "sjmm_suf_2024_jsonl"
)

EXPOSURE_DIR = ROOT / "derived" / "exposures"
EXPOSURE_DIR.mkdir(parents=True, exist_ok=True)
DERIVED_DATA_PATH = ROOT / "derived" / "sjmm_ai_exposure.jsonl"

In [10]:
import os
os.path.isdir(INDEXES_DIR)

True

In [11]:
ROOT

PosixPath('/Users/miguel/Documents/Master Thesis/Thesis')

In [12]:
INDEXES_DIR

PosixPath('/Users/miguel/Documents/Master Thesis/Thesis/Aditional datasets/indexes')

In [13]:
"""Return a dataframe indexed by ISCO with exposure info (SOC10-based)."""
soc_exposure = pd.read_excel(
    INDEXES_DIR / "General AIOE and AIIE.xlsx",
    sheet_name="General AIOE (SOC)",
    usecols=["SOC Code", "AIOE"],
    dtype=str,
).dropna(subset=["SOC Code", "AIOE"])
soc_exposure["SOC Code"] = soc_exposure["SOC Code"].str.strip()
soc_exposure["AIOE"] = pd.to_numeric(soc_exposure["AIOE"], errors="coerce")

crosswalk = pd.read_csv(
    CROSSWALKS_DIR / "SOC_ISCO_Crosswalk .csv",
    usecols=["2010 SOC Code", "ISCO-08 Code", "part"],
    dtype=str,
).dropna(subset=["2010 SOC Code", "ISCO-08 Code"])

crosswalk["2010 SOC Code"] = crosswalk["2010 SOC Code"].str.strip()
crosswalk["ISCO-08 Code"] = (
    crosswalk["ISCO-08 Code"].str.replace(".", "", regex=False).str.strip().str.zfill(4)
)

merged = crosswalk.merge(
    soc_exposure,
    left_on="2010 SOC Code",
    right_on="SOC Code",
    how="left",
)

def summarise(group: pd.DataFrame) -> pd.Series:
    soc_codes = sorted(group["2010 SOC Code"].unique())
    available = group.dropna(subset=["AIOE"])
    matched_soc = sorted(available["2010 SOC Code"].unique())
    missing_soc = sorted(set(soc_codes) - set(matched_soc))
    aioe = float(available["AIOE"].mean()) if not available.empty else np.nan
    return pd.Series(
        {
            "aioe": aioe,
            "soc_codes": soc_codes,
            "matched_soc_codes": matched_soc,
            "missing_soc_codes": missing_soc,
            "partial_mapping_count": int((group["part"] == "*").sum()),
        }
    )

occupation_df = merged.groupby("ISCO-08 Code").apply(
    summarise, include_groups=False
)
occupation_df.index.name = "ISCO-08 Code"

occ_missing = {
    isco: codes for isco, codes in occupation_df["missing_soc_codes"].items() if codes
}

In [14]:
occupation_df

,aioe,soc_codes,matched_soc_codes,missing_soc_codes,partial_mapping_count
ISCO-08 Code,,,,,
0110,NaN,"[55-1011, 55-1012, 55-1013, 55-1014, 55-1015, ...",[],"[55-1011, 55-1012, 55-1013, 55-1014, 55-1015, ...",0
0210,NaN,"[55-2011, 55-2012, 55-2013]",[],"[55-2011, 55-2012, 55-2013]",0
0211,1.140157,[19-2099],[19-2099],[],0
0310,NaN,"[55-3011, 55-3012, 55-3013, 55-3014, 55-3015, ...",[],"[55-3011, 55-3012, 55-3013, 55-3014, 55-3015, ...",0
0315,0.484574,[53-2022],[53-2022],[],0
...,...,...,...,...,...
9621,-1.115564,"[39-6011, 43-5021]","[39-6011, 43-5021]",[],1
9622,-0.983681,"[37-3019, 49-9071, 49-9098, 49-9099]","[49-9071, 49-9098, 49-9099]",[37-3019],1
9623,-0.847888,"[43-5041, 49-9091]","[43-5041, 49-9091]",[],0


In [35]:
crosswalk[crosswalk["2010 SOC Code"] == "11-1011"]

,2010 SOC Code,part,ISCO-08 Code
0,11-1011,*,1112
1,11-1011,*,1113
2,11-1011,*,1120


In [37]:
merged

,2010 SOC Code,part,ISCO-08 Code,SOC Code,AIOE
0,11-1011,*,1112,11-1011,1.334246
1,11-1011,*,1113,11-1011,1.334246
2,11-1011,*,1120,11-1011,1.334246
3,11-1021,*,1112,11-1021,0.574877
4,11-1021,*,1114,11-1021,0.574877
...,...,...,...,...,...
1120,55-3015,NaN,0310,NaN,NaN
1121,55-3016,NaN,0310,NaN,NaN
1122,55-3017,NaN,0310,NaN,NaN
1123,55-3018,NaN,0310,NaN,NaN


In [39]:
merged[merged["ISCO-08 Code"] == "9622"]

,2010 SOC Code,part,ISCO-08 Code,SOC Code,AIOE
512,37-3019,*,9622,NaN,NaN
853,49-9071,NaN,9622,49-9071,-1.014408
862,49-9098,NaN,9622,49-9098,-1.346555
863,49-9099,NaN,9622,49-9099,-0.590079


In [40]:
occ_missing

{'0110': ['55-1011',
  '55-1012',
  '55-1013',
  '55-1014',
  '55-1015',
  '55-1016',
  '55-1017',
  '55-1019'],
 '0210': ['55-2011', '55-2012', '55-2013'],
 '0310': ['55-3011',
  '55-3012',
  '55-3013',
  '55-3014',
  '55-3015',
  '55-3016',
  '55-3017',
  '55-3018',
  '55-3019'],
 '1111': ['11-1031'],
 '1113': ['11-1031'],
 '2131': ['19-1099'],
 '2163': ['27-1029'],
 '2261': ['29-1029'],
 '2269': ['29-1129'],
 '2310': ['25-1069', '25-1199'],
 '2359': ['25-9099'],
 '2635': ['21-1019', '21-1029', '21-1099'],
 '2659': ['27-2099'],
 '3118': ['17-3019'],
 '3314': ['15-2099'],
 '3339': ['41-9099'],
 '3411': ['23-2099'],
 '3413': ['21-2099'],
 '3435': ['27-1019', '27-2099', '27-3099', '27-4099', '39-3099'],
 '3521': ['27-4099'],
 '4212': ['39-3019'],
 '4213': ['41-9099', '43-3099'],
 '4229': ['43-2099', '43-4199'],
 '4312': ['43-3099'],
 '4419': ['43-4199', '43-9199'],
 '5120': ['35-2019'],
 '5152': ['39-9099'],
 '5161': ['27-2099'],
 '5162': ['39-9099'],
 '5169': ['39-9099'],
 '5249': ['41

In [4]:
from preprocess_naics_exposure import load_clean_naics_exposure

"""Return dataframe keyed by NOGA-2 with exposure and contributions."""
naics_exposure = load_clean_naics_exposure(
    INDEXES_DIR / "General AIOE and AIIE.xlsx",
    CROSSWALKS_DIR / "2017_NAICS_to_ISIC_4.csv",
)

naics_isic = pd.read_csv(
    CROSSWALKS_DIR / "2017_NAICS_to_ISIC_4.csv",
    dtype=str,
)
naics_isic["naics_clean"] = (
    naics_isic["2017\nNAICS\nUS  "]
    .astype(str)
    .str.replace(".", "", regex=False)
    .str.strip()
)
naics_isic["isic_clean"] = (
    naics_isic["ISIC 4.0"].astype(str).str.replace(".", "", regex=False).str.strip()
)
naics_isic = naics_isic[
    naics_isic["naics_clean"].str.fullmatch(r"\d+")
    & naics_isic["isic_clean"].str.fullmatch(r"\d+")
]
naics_isic["naics4"] = naics_isic["naics_clean"].str[:4]
naics_isic = naics_isic[["naics4", "isic_clean"]].drop_duplicates()

matched = naics_exposure.merge(
    naics_isic,
    left_on="NAICS",
    right_on="naics4",
    how="left",
)

naics_gaps: Dict[str, str] = {}
no_isic = matched[matched["isic_clean"].isna()]["NAICS"].unique()
for code in sorted(no_isic):
    naics_gaps[code] = "no ISIC match"

matched = matched.dropna(subset=["isic_clean"]).copy()
if matched.empty:
    print('merged empty---------------')

matched["isic_clean"] = (
    matched["isic_clean"].astype(str).str.replace(".", "", regex=False).str.strip()
)
matched = matched.drop_duplicates(subset=["NAICS", "isic_clean"])

isic_nace = pd.read_csv(
    CROSSWALKS_DIR / "ISIC4_NACE2.csv",
    usecols=["ISIC4code", "NACE2code"],
    dtype=str,
).dropna(subset=["ISIC4code", "NACE2code"])
isic_nace["isic_clean"] = (
    isic_nace["ISIC4code"].str.replace(".", "", regex=False).str.strip()
)
isic_nace["noga_2"] = (
    isic_nace["NACE2code"].str.split(".").str[0].str.strip().str.upper()
)
mask_numeric = isic_nace["noga_2"].str.fullmatch(r"\d+")
isic_nace.loc[mask_numeric, "noga_2"] = (
    isic_nace.loc[mask_numeric, "noga_2"].str.zfill(2)
)

merged = matched.merge(
    isic_nace[["isic_clean", "noga_2"]].drop_duplicates(),
    on="isic_clean",
    how="left",
)

naics_with_noga = set(merged[merged["noga_2"].notna()]["NAICS"].unique())
naics_without_noga = sorted(set(matched["NAICS"]) - naics_with_noga)
for code in naics_without_noga:
    naics_gaps.setdefault(code, "no NOGA match")

merged = merged[merged["noga_2"].notna()].copy()
if merged.empty:
    print('merged empty---------------')

merged = merged.drop_duplicates(subset=["NAICS", "isic_clean", "noga_2"])

contributions = (
    merged.groupby(["noga_2", "NAICS"])
    .agg(
        aiie=("AIIE", "first"),
        isic_codes=("isic_clean", lambda s: sorted(set(s))),
    )
    .reset_index()
)

noga_aiie = contributions.groupby("noga_2")["aiie"].mean().to_frame(name="aiie")
noga_aiie["aiie"] = noga_aiie["aiie"].astype(float)

noga_aiie["contributions"] = contributions.groupby("noga_2").apply(
    lambda df: [
        {
            "naics": row.NAICS,
            "aiie": float(row.aiie),
            "isic_codes": row.isic_codes,
        }
        for row in df.itertuples(index=False)
    ],
    include_groups=False,
)
noga_aiie["naics_codes"] = noga_aiie["contributions"].apply(
    lambda lst: sorted({item["naics"] for item in lst})
)
noga_aiie["isic_codes"] = noga_aiie["contributions"].apply(
    lambda lst: sorted({code for item in lst for code in item["isic_codes"]})
)

#return noga_aiie, naics_gaps

In [78]:
from preprocess_naics_exposure import load_clean_naics_exposure

"""Return dataframe keyed by NOGA-2 with exposure and contributions."""
naics_exposure = load_clean_naics_exposure(
    INDEXES_DIR / "General AIOE and AIIE.xlsx",
    CROSSWALKS_DIR / "2017_NAICS_to_ISIC_4.csv",
)

naics_isic = pd.read_csv(
    CROSSWALKS_DIR / "2017_NAICS_to_ISIC_4.csv",
    dtype=str,
)
naics_isic["naics_clean"] = (
    naics_isic["2017\nNAICS\nUS  "]
    .astype(str)
    .str.replace(".", "", regex=False)
    .str.strip()
)
naics_isic["isic_clean"] = (
    naics_isic["ISIC 4.0"].astype(str).str.replace(".", "", regex=False).str.strip()
)
naics_isic = naics_isic[
    naics_isic["naics_clean"].str.fullmatch(r"\d+")
    & naics_isic["isic_clean"].str.fullmatch(r"\d+")
]
naics_isic["naics4"] = naics_isic["naics_clean"].str[:4]
naics_isic = naics_isic[["naics4", "isic_clean"]].drop_duplicates()

matched = naics_exposure.merge(
    naics_isic,
    left_on="NAICS",
    right_on="naics4",
    how="left",
)

naics_gaps: Dict[str, str] = {}
no_isic = matched[matched["isic_clean"].isna()]["NAICS"].unique()
for code in sorted(no_isic):
    naics_gaps[code] = "no ISIC match"

matched = matched.dropna(subset=["isic_clean"]).copy()
if matched.empty:
    print('merged empty---------------')

matched["isic_clean"] = (
    matched["isic_clean"].astype(str).str.replace(".", "", regex=False).str.strip()
)
matched = matched.drop_duplicates(subset=["NAICS", "isic_clean"])

isic_nace = pd.read_csv(
    CROSSWALKS_DIR / "ISIC4_NACE2.csv",
    usecols=["ISIC4code", "NACE2code"],
    dtype=str,
).dropna(subset=["ISIC4code", "NACE2code"])
isic_nace["isic_clean"] = (
    isic_nace["ISIC4code"].str.replace(".", "", regex=False).str.strip()
)
isic_nace["noga_2"] = (
    isic_nace["NACE2code"].str.split(".").str[0].str.strip().str.upper()
)

mask_numeric = isic_nace["noga_2"].str.fullmatch(r"\d+")
isic_nace.loc[mask_numeric, "noga_2"] = (
    isic_nace.loc[mask_numeric, "noga_2"].str.zfill(2)
)

merged = matched.merge(
    isic_nace[["isic_clean", "noga_2"]].drop_duplicates(),
    on="isic_clean",
    how="left",
)

naics_with_noga = set(merged[merged["noga_2"].notna()]["NAICS"].unique())
naics_without_noga = sorted(set(matched["NAICS"]) - naics_with_noga)
for code in naics_without_noga:
    naics_gaps.setdefault(code, "no NOGA match")

merged = merged[merged["noga_2"].notna()].copy()
if merged.empty:
    print('merge is empty---------------------')

merged = merged.drop_duplicates(subset=["NAICS", "isic_clean", "noga_2"])

contributions = (
    merged.groupby(["noga_2", "NAICS"])
    .agg(
        aiie=("AIIE", "first"),
        isic_codes=("isic_clean", lambda s: sorted(set(s))),
        isic_count=("isic_clean", lambda s: len(set(s))),
    )
    .reset_index()
)

mean_naics = contributions.groupby("noga_2")["aiie"].mean()
weighted = contributions.groupby("noga_2").apply(
    lambda df: (df["aiie"] * df["isic_count"]).sum() / df["isic_count"].sum(),
    include_groups=False,
)
noga_aiie = pd.concat(
    [mean_naics.rename("aiie"), weighted.rename("aiie_weighted")], axis=1
)
noga_aiie["aiie"] = noga_aiie["aiie"].astype(float)
noga_aiie["aiie_weighted"] = noga_aiie["aiie_weighted"].astype(float)

noga_aiie["contributions"] = contributions.groupby("noga_2").apply(
    lambda df: [
        {
            "naics": row.NAICS,
            "aiie": float(row.aiie),
            "isic_codes": row.isic_codes,
            "isic_count": int(row.isic_count),
        }
        for row in df.itertuples(index=False)
    ],
    include_groups=False,
)
noga_aiie["naics_codes"] = noga_aiie["contributions"].apply(
    lambda lst: sorted({item["naics"] for item in lst})
)
noga_aiie["isic_codes"] = noga_aiie["contributions"].apply(
    lambda lst: sorted({code for item in lst for code in item["isic_codes"]})
)

In [74]:
contributions.head(20)

,noga_2,NAICS,aiie,isic_codes,isic_count
0,01,1151,-2.165304,"[0161, 0162, 0163, 0164]",4
1,01,1152,-1.149307,[0162],1
2,02,1133,-1.360161,"[0220, 0240]",2
3,03,3117,-1.258223,[0311],1
4,05,2111,0.668615,[0510],1
5,05,2121,-1.146242,"[0510, 0520]",2
6,06,2111,0.668615,"[0610, 0620]",2
7,07,2122,-0.892403,"[0710, 0721, 0729]",3
8,08,2123,-1.037903,"[0810, 0891, 0892, 0893, 0899]",5
9,08,3119,-0.853891,[0893],1


In [83]:
noga_aiie

,aiie,aiie_weighted,contributions,naics_codes,isic_codes
noga_2,,,,,
01,-1.657306,-1.962105,"[{'naics': '1151', 'aiie': -2.165304, 'isic_co...","[1151, 1152]","[0161, 0162, 0163, 0164]"
02,-1.360161,-1.360161,"[{'naics': '1133', 'aiie': -1.360161, 'isic_co...",[1133],"[0220, 0240]"
03,-1.258223,-1.258223,"[{'naics': '3117', 'aiie': -1.258223, 'isic_co...",[3117],[0311]
05,-0.238813,-0.541290,"[{'naics': '2111', 'aiie': 0.6686154, 'isic_co...","[2111, 2121]","[0510, 0520]"
06,0.668615,0.668615,"[{'naics': '2111', 'aiie': 0.6686154, 'isic_co...",[2111],"[0610, 0620]"
...,...,...,...,...,...
92,-0.519076,-0.519076,"[{'naics': '7132', 'aiie': -0.1355364, 'isic_c...","[7132, 7211]",[9200]
93,-0.090274,-0.373734,"[{'naics': '7111', 'aiie': 0.1677087, 'isic_co...","[7111, 7112, 7113, 7131, 7139, 8139]","[9311, 9312, 9319, 9321, 9329]"
94,0.978263,1.148076,"[{'naics': '8131', 'aiie': 0.9662193, 'isic_co...","[8131, 8132, 8133, 8134, 8139]","[9411, 9412, 9420, 9491, 9492, 9499]"


In [70]:
noga_aiie

,aiie
noga_2,
01,-1.657306
02,-1.360161
03,-1.258223
05,-0.238813
06,0.668615
...,...
92,-0.519076
93,-0.090274
94,0.978263


In [54]:
merged

,NAICS,AIIE,naics4,isic_clean,noga_2
0,1133,-1.360161,1133,0220,02
1,1133,-1.360161,1133,0240,02
2,1133,-1.360161,1133,1610,16
3,1151,-2.165304,1151,0163,01
4,1151,-2.165304,1151,0161,01
...,...,...,...,...,...
862,8139,1.360341,8139,9412,94
863,8139,1.360341,8139,9420,94
864,8139,1.360341,8139,9492,94
865,8139,1.360341,8139,9319,93


In [50]:
matched

,NAICS,AIIE,naics4,isic_clean
0,1133,-1.360161,1133,0220
1,1133,-1.360161,1133,0240
2,1133,-1.360161,1133,1610
3,1151,-2.165304,1151,0163
4,1151,-2.165304,1151,0161
...,...,...,...,...
862,8139,1.360341,8139,9412
863,8139,1.360341,8139,9420
864,8139,1.360341,8139,9492
865,8139,1.360341,8139,9319
